# Scenario 1 – GRAPE Simulation (Notebook Version)
Distributed Task Allocation for Autonomous Airspace Sharing

In [ ]:
# 기본 패키지 로딩
import random
import math
import matplotlib.pyplot as plt

NUM_AGENTS = 5
NUM_TASKS = 5
MAX_STEPS = 10

ALPHA = 1.0
BETA = 1.0
GAMMA = 1.0
DELTA = 1.0

In [ ]:
# Task 클래스
class Task:
    def __init__(self, tid, location):
        self.tid = tid
        self.location = location
        self.coalition = []

    def update_coalition(self, agent):
        if agent not in self.coalition:
            self.coalition.append(agent)

    def reset_coalition(self):
        self.coalition = []

In [ ]:
# Agent 클래스
class Agent:
    def __init__(self, aid, position):
        self.aid = aid
        self.position = position
        self.assigned_task = None

    def evaluate_tasks(self, tasks):
        utilities = []
        for task in tasks:
            d_ij = self.distance(task.location)
            p_collision = len(task.coalition) / 10
            wait_time = random.uniform(1, 5)
            utility = (
                ALPHA / d_ij +
                BETA * (1 - p_collision) +
                GAMMA / wait_time -
                DELTA * len(task.coalition)
            )
            utilities.append((utility, task))
        utilities.sort(reverse=True, key=lambda x: x[0])
        return utilities[0][1]

    def distance(self, task_location):
        dx = self.position[0] - task_location[0]
        dy = self.position[1] - task_location[1]
        return math.sqrt(dx**2 + dy**2)

In [ ]:
# 시뮬레이션 함수 + 시각화
def run_simulation():
    agents = [Agent(aid=i, position=(0, i)) for i in range(NUM_AGENTS)]
    tasks = [Task(tid=j, location=(9, j)) for j in range(NUM_TASKS)]
    
    plt.figure(figsize=(8, 6))
    for step in range(MAX_STEPS):
        for task in tasks:
            task.reset_coalition()
        for agent in agents:
            chosen_task = agent.evaluate_tasks(tasks)
            chosen_task.update_coalition(agent)
            agent.assigned_task = chosen_task
        # 시각화
        plt.clf()
        for agent in agents:
            x0, y0 = agent.position
            x1, y1 = agent.assigned_task.location
            plt.plot([x0, x1], [y0, y1], 'k--', alpha=0.4)
            plt.scatter(x0, y0, c='blue', label='Agent' if step == 0 else "")
            plt.scatter(x1, y1, c='red', label='Task' if step == 0 else "")
        plt.title(f"Step {step + 1}")
        plt.xlim(-1, 11)
        plt.ylim(-1, max(NUM_AGENTS, NUM_TASKS) + 1)
        plt.pause(0.5)
    plt.legend()
    plt.show()

run_simulation()